In [ ]:
import oracledb
from google.cloud.bigquery import Client
import os, json
from google.cloud import secretmanager

In [ ]:
def set_secrets_as_envs():
  secrets = secretmanager.SecretManagerServiceClient()
  resource_name = f"{os.environ['KNADA_TEAM_SECRET']}/versions/latest"
  secret = secrets.access_secret_version(name=resource_name)
  secret_str = secret.payload.data.decode('UTF-8')
  secrets = json.loads(secret_str)
  os.environ.update(secrets)

In [ ]:
def oracle_secrets():
  set_secrets_as_envs()
  return dict(
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    host = os.getenv('DBT_ORCL_HOST'),
    service = os.getenv('DBT_ORCL_SERVICE'),
    project_id = os.getenv('BS_PROJECT_ID'),
    table_uri = os.getenv('BS_TABLE_URI'),  
    encoding="UTF-8",
    nencoding="UTF-8"
  )

oracle_secrets = oracle_secrets()

In [ ]:
def get_data_from_table(max_id):
    PROJECT_ID = oracle_secrets['project_id'] 
    TABLE_URI = oracle_secrets['table_uri'] 

    client = Client(project=PROJECT_ID)
    job = client.query(f"SELECT * FROM `{TABLE_URI}` WHERE id > {max_id}")
    df = job.to_dataframe()
    
    # some data cleaning
    char_to_replace = {'≥' : '>=', '≤' : '<='}
    for old, new in char_to_replace.items():
        df['sats_beskrivelse'] = df['sats_beskrivelse'].str.replace(old, new)
    
    return df

In [ ]:
def send_data():
    user = oracle_secrets['user'] + '[DVH_FAM_HM]'
    dsn_tns = oracledb.makedsn(oracle_secrets['host'], 1521, service_name = oracle_secrets['service'])
    
    with oracledb.connect(user=user, password = oracle_secrets['password'], dsn=dsn_tns) as conn:
        with conn.cursor() as cursor:
            sql = """select max(id) as maximun from brillestonad"""
            cursor.execute(sql)
            result = cursor.fetchall()
            max_id = result[0][0]

            if max_id is None:
                df = get_data_from_table(0)
            else:
                df = get_data_from_table(max_id)
            
            if len(df) > 0:
                rows = [tuple(x) for x in df.values]
                cursor.executemany('''INSERT INTO brillestonad_test (ID,FNR_BARN,FNR_INNSENDER,ORGNR,BESTILLINGSDATO,BRILLEPRIS,BESTILLINGSREFERANSE,
                                        BEHANDLINGSRESULTAT,SATS,SATS_BELOP,SATS_BESKRIVELSE,BELOP,OPPRETTET,KILDE) 
                                         VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14)''',rows)
                conn.commit()

In [ ]:
send_data()